In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import jax.numpy as jnpra
from scalaremlp.reps import Scalar, Vector, T
from torch.utils.data import Dataset
from scalaremlp.utils import export
from scalaremlp.groups import SO, O, Trivial, Lorentz, RubiksCube, Cube
from functools import partial
import itertools
from jax import vmap, jit
from objax import Module

In [ ]:
@export
class Inertia(Dataset):
    def __init__(self, N=1024, k=5):
        super().__init__()
        self.dim = (1 + 3) * k
        self.X = torch.randn(N, self.dim)
        self.X[:, :k] = F.softplus(self.X[:, :k])  # Masses
        mi = self.X[:, :k]
        ri = self.X[:, k:].reshape(-1, k, 3)
        I = torch.eye(3)
        r2 = (ri ** 2).sum(-1)[..., None, None]
        inertia = (mi[:, :, None, None] * (r2 * I - ri[..., None] * ri[..., None, :])).sum(1)
        self.Y = inertia.reshape(-1, 9)
        self.rep_in = k * Scalar + k * Vector
        self.rep_out = T(2)
        self.symmetry = O(3)
        self.X = self.X.numpy()
        self.Y = self.Y.numpy()
        # One has to be careful computing offset and scale in a way so that standardizing
        # does not violate equivariance
        Xmean = self.X.mean(0)
        Xmean[k:] = 0
        Xstd = np.zeros_like(Xmean)
        Xstd[:k] = np.abs(self.X[:, :k]).mean(0)  # .std(0)
        # Xstd[k:] = (np.sqrt((self.X[:,k:].reshape(N,k,3)**2).mean((0,2))[:,None]) + np.zeros((k,3))).reshape(k*3)
        Xstd[k:] = (np.abs(self.X[:, k:].reshape(N, k, 3)).mean((0, 2))[:, None] + np.zeros((k, 3))).reshape(k * 3)
        Ymean = 0 * self.Y.mean(0)
        # Ystd = np.sqrt(((self.Y-Ymean)**2).mean((0,1)))+ np.zeros_like(Ymean)
        Ystd = np.abs(self.Y - Ymean).mean((0, 1)) + np.zeros_like(Ymean)
        self.stats = 0, 1, 0, 1  # Xmean,Xstd,Ymean,Ystd
        self.symname = "O3equivariant"

In [25]:
# with N data points, for each one,
N=1024
k=5
dim=4*k

# derive data by random numbers
X = torch.randn(N, dim)
# print(X)

# ensure masses are positive(first five columns)
X[:, :k] = F.softplus(X[:, :k])  # Masses
# print(X)

# mass(first five columns)
mi = X[:, :k]

# shape: 1024, 5, 3, position of each mass
ri = X[:, k:].reshape(-1, k, 3)


I = torch.eye(3)
# print(ri**2)
# summation over each vector of position
r2 = (ri ** 2).sum(-1)[..., None, None]
inertia = (mi[:, :, None, None] * (r2 * I - ri[..., None] * ri[..., None, :])).sum(1)
print(inertia.shape)

Y = inertia.reshape(-1, 9)
rep_in = k * Scalar + k * Vector
rep_out = T(2)
symmetry = O(3)

torch.Size([1024, 3, 3])


In [26]:
O(3)

O(3)